# Tagging efficiency study at 4GeV beam energy

Analysis of results from the 500M electron simulation of the GlueX photon source at 4GeV electron beam energy. There are 25 of these beam electrons per 4ns beam bucket per nA of electron beam current. This entire simulation sample represents 80ms of beamtime at 1nA electron beam current.

In [1]:
import sys
import random
import ROOT
%jsroot on
import os
os.chdir("/srv/jupyter/gxtwist studies")
import pyxrootd.client as xclient
import numpy as np
#%pip install --user "gluex.jupyroot>=1.0.10"
from gluex.jupyroot.treeview import treeview

In [2]:
import dask.distributed
import dask
dclient = dask.distributed.Client(n_workers=50, threads_per_worker=1, dashboard_address='0.0.0.0:8786')

In [3]:
xrdurl = "root://cn445.storrs.hpc.uconn.edu"
xrdpath = "/Gluex/resilient/simulation/tag4GeV-10-2024/"
rfile1 = ROOT.TFile.Open(xrdurl + xrdpath + "tag4GeV_100.root")
treekey = {'tagm': 'h1', 'tagh': 'h2', 'det7': 'h5', 'det8': 'h6'}
for i in range(7):
    treekey[f"det{i}"] = f"h{i+7}"
tree = {key: rfile1.Get(treekey[key]) for key in treekey}
chain = {key: ROOT.TChain(tree[key].GetName(), tree[key].GetTitle()) for key in treekey}
xfs = xclient.FileSystem(xrdurl)
for f in xfs.dirlist(xrdpath)[1]['dirlist']:
    if "tag4GeV_" in f['name']:
        [chain[key].Add(xrdurl + xrdpath + f['name']) for key in treekey]
[print("chain", key, "created with", chain[key].GetNtrees(), "files") for key in ('tagm',)]
tree['tagm'].Print()
tview = {key: treeview(chain[key], f"tag4GeV.root/{key}") for key in treekey}
tview['det0'].enable_dask_cluster(dclient)

chain tagm created with 997 files
******************************************************************************
*Tree    :h1        : microscope                                             *
*Entries :        6 : Total =            7338 bytes  File  Size =       2503 *
*        :          : Tree compression factor =   1.10                       *
******************************************************************************
*Br    0 :eventno   : eventno/I                                              *
*Entries :        6 : Total  Size=        589 bytes  File Size  =         96 *
*Baskets :        1 : Basket Size=      64000 bytes  Compression=   1.00     *
*............................................................................*
*Br    1 :kind      : kind/I                                                 *
*Entries :        6 : Total  Size=        574 bytes  File Size  =         93 *
*Baskets :        1 : Basket Size=      64000 bytes  Compression=   1.00     *
*.................

In [4]:
MeV = 1e-3 #GeV
def scale_errors(h1d, factor=1):
    """
    Rescale the errors on the contents of a 1d histogram by factor.
    """
    for i in range(h1d.GetNbinsX()):
        h1d.SetBinError(i+1, h1d.GetBinError(i+1) * factor)

## 1. Rates in the tagging counters

In [5]:
def tagm_rate_hinit():
    h = {}
    h['htagm'] = ROOT.TH1D("htagm", "hits per microscope column", 1200, 730, 850)
    h['htagm'].GetXaxis().SetTitle("distance along focal plane (cm)")
    h['htagm'].GetYaxis().SetTitle("counts")
    h['htagm_dEmin'] = ROOT.TH1D("htagm_dEmin", "hits per microscope column, dE>dEmin", 1200, 730, 850)
    h['htagm_dEmin'].GetXaxis().SetTitle("distance along focal plane (cm)")
    h['htagm_dEmin'].GetYaxis().SetTitle("counts")
    h['htagm_dEmin'].SetLineColor(ROOT.kOrange + 7)
    h['htagmdE'] = ROOT.TH1D("htagmdE", "microscope pulse height", 100, 0, 10)
    h['htagmdE'].GetXaxis().SetTitle("#Delta E (MeV)")
    h['htagmdE'].GetYaxis().SetTitle("counts")
    h['htagmdE_dEmin'] = ROOT.TH1D("htagmdE_dEmin", "microscope pulse height, dE>dEmin", 100, 0, 10)
    h['htagmdE_dEmin'].GetXaxis().SetTitle("#Delta E (MeV)")
    h['htagmdE_dEmin'].GetYaxis().SetTitle("counts")
    h['htagmdE_dEmin'].SetLineColor(ROOT.kOrange + 7)
    return h
tagm_tags = {}
def tagm_rate_hfill(row, histos, dEmin=2*MeV):
    histos['htagm'].Fill((row.xin[2]+row.xout[2])/2)
    histos['htagmdE'].Fill(row.desum/MeV)
    if row.desum > dEmin:
        histos['htagm_dEmin'].Fill((row.xin[2]+row.xout[2])/2)
        histos['htagmdE_dEmin'].Fill(row.desum/MeV)
        evid = f"{row.GetFileNumber()}:{row.eventno}"
        if not evid in tagm_tags:
            tagm_tags[evid] = []
        tagm_tags[evid].append([row.xin[2], row.xin[3], row.pin[3]]) 
    return histos

def tagh_rate_hinit():
    h = {}
    h['htagh'] = ROOT.TH1D("htagh", "hits per hodooscope counter", 1200, 300, 1500)
    h['htagh'].GetXaxis().SetTitle("distance along focal plane (cm)")
    h['htagh'].GetYaxis().SetTitle("counts")
    h['htagh_dEmin'] = ROOT.TH1D("htagh_dEmin", "hits per hodooscope counter, dE>dEmin", 1200, 300, 1500)
    h['htagh_dEmin'].GetXaxis().SetTitle("distance along focal plane (cm)")
    h['htagh_dEmin'].GetYaxis().SetTitle("counts")
    h['htagh_dEmin'].SetLineColor(ROOT.kOrange + 7)
    h['htaghdE'] = ROOT.TH1D("htaghdE", "hodoscope pulse height", 100, 0, 10)
    h['htaghdE'].GetXaxis().SetTitle("#Delta E (MeV)")
    h['htaghdE'].GetYaxis().SetTitle("counts")
    h['htaghdE_dEmin'] = ROOT.TH1D("htaghdE_dEmin", "hodoscope pulse height, dE>dEmin", 100, 0, 10)
    h['htaghdE_dEmin'].GetXaxis().SetTitle("#Delta E (MeV)")
    h['htaghdE_dEmin'].GetYaxis().SetTitle("counts")
    h['htaghdE_dEmin'].SetLineColor(ROOT.kOrange + 7)
    return h
tagh_tags = {}
def tagh_rate_hfill(row, histos, dEmin=0.4*MeV):
    histos['htagh'].Fill((row.xin[2]+row.xout[2])/2)
    histos['htaghdE'].Fill(row.desum/MeV)
    if row.desum > dEmin:
        histos['htagh_dEmin'].Fill((row.xin[2]+row.xout[2])/2)
        histos['htaghdE_dEmin'].Fill(row.desum/MeV)
        evid = f"{row.GetFileNumber()}:{row.eventno}"
        if not evid in tagh_tags:
            tagh_tags[evid] = []
        tagh_tags[evid].append([row.xin[2], row.xin[3], row.pin[3]]) 
    return histos

def det7_rate_hinit():
    h = {}
    h['hdet7'] = ROOT.TH1D("hdet7", "e-hits at focal plane", 1200, 300, 1500)
    h['hdet7'].GetXaxis().SetTitle("distance along focal plane (cm)")
    h['hdet7'].GetYaxis().SetTitle("counts")
    h['hdet7_Emin'] = ROOT.TH1D("hdet7_Emin", "e-hits per hodooscope counter, E>Emin", 1200, 300, 1500)
    h['hdet7_Emin'].GetXaxis().SetTitle("distance along focal plane (cm)")
    h['hdet7_Emin'].GetYaxis().SetTitle("counts")
    h['hdet7_Emin'].SetLineColor(ROOT.kOrange + 7)
    h['hdet7E'] = ROOT.TH1D("hdet7E", "e-energy at focal plane", 100, -2, 4)
    h['hdet7E'].GetXaxis().SetTitle("log10(E/MeV)")
    h['hdet7E'].GetYaxis().SetTitle("counts")
    h['hdet7E_Emin'] = ROOT.TH1D("hdet7E_Emin", "e-energy at focal plane, E>Emin", 100, -2, 4)
    h['hdet7E_Emin'].GetXaxis().SetTitle("log(E/MeV)")
    h['hdet7E_Emin'].GetYaxis().SetTitle("counts")
    h['hdet7E_Emin'].SetLineColor(ROOT.kOrange + 7)
    return h
det7_tags = {}
def det7_rate_hfill(row, histos, Emin=100*MeV):
    histos['hdet7'].Fill((row.xin[2]+row.xout[2])/2)
    histos['hdet7E'].Fill(np.log10(row.pin[3]/MeV))
    if row.kind == 3 and row.pin[3] > Emin:
        histos['hdet7_Emin'].Fill((row.xin[2]+row.xout[2])/2)
        histos['hdet7E_Emin'].Fill(np.log10(row.pin[3]/MeV))
        evid = f"{row.GetFileNumber()}:{row.eventno}"
        if not evid in det7_tags:
            det7_tags[evid] = []
        det7_tags[evid].append([row.xin[2], row.xin[3], row.pin[3]]) 
    return histos

u = tview['tagm'].declare_histograms("microscope hits", tagm_rate_hinit, tagm_rate_hfill)
u = tview['tagh'].declare_histograms("hodoscope hits", tagh_rate_hinit, tagh_rate_hfill)
u = tview['det7'].declare_histograms("focal plane hits", det7_rate_hinit, det7_rate_hfill)

In [6]:
tview['tagm'].fill_histograms()
tview['tagh'].fill_histograms()
tview['det7'].fill_histograms()
n = tview['tagm'].draw([[['htagm', 'htagm_dEmin'], ['htagmdE', 'htagmdE_dEmin']]], stats="ien")
n = tview['tagh'].draw([[['htagh', 'htagh_dEmin'], ['htaghdE', 'htaghdE_dEmin']]], stats="ien")
n = tview['det7'].draw([[['hdet7', 'hdet7_Emin'], ['hdet7E', 'hdet7E_Emin']]], stats="ien")
print("Number of microscope hits over threshold was", tview['tagm'].get('htagmdE_dEmin').GetEntries(), "from", len(tagm_tags), "events")
print("Number of hodoscope hits over threshold was", tview['tagh'].get('htaghdE_dEmin').GetEntries(), "from", len(tagh_tags), "events")
print("Number of focal plane electrons with E>100MeV was", tview['det7'].get('hdet7E_Emin').GetEntries(), "from", len(det7_tags), "events")

fill_histograms read a total of 983 tree files, 10856.0 records
fill_histograms read a total of 997 tree files, 28576.0 records
fill_histograms read a total of 997 tree files, 514450.0 records
Number of microscope hits over threshold was 2524.0 from 0 events
Number of hodoscope hits over threshold was 17178.0 from 0 events
Number of focal plane electrons with E>100MeV was 42057.0 from 0 events


## 2. Photon beam at the primary collimator

In [7]:
def beam_photon_hinit():
    h = {}
    h['hcolspot1'] = ROOT.TH2D('hcolspot1', "photon beam at the primary collimator", 200, -5, 5, 200, -5, 5)
    h['hcolspot1'].GetXaxis().SetTitle("x (cm)")
    h['hcolspot1'].GetYaxis().SetTitle("y (cm)")
    h['hcolspot2'] = ROOT.TH2D('hcolspot2', "photon beam after the primary collimator", 200, -5, 5, 200, -5, 5)
    h['hcolspot2'].GetXaxis().SetTitle("x (cm)")
    h['hcolspot2'].GetYaxis().SetTitle("y (cm)")
    h['hcolspec1'] = ROOT.TH1D('hcolspec1', "photon beam spectrum at the primary collimator", 200, -4, 4)
    h['hcolspec1'].GetXaxis().SetTitle("log_{10}(E#gamma/MeV)")
    h['hcolspec1'].GetYaxis().SetTitle("counts")
    h['hcolspec2'] = ROOT.TH1D('hcolspec2', "photon beam spectrum after the primary collimator", 200, -4, 4)
    h['hcolspec2'].GetXaxis().SetTitle("log_{10}(E#gamma/MeV)")
    h['hcolspec2'].GetYaxis().SetTitle("counts")
    return h
def beam_photon_hfill(row, histos, colimz=7600, colimr=0.25):
    if row.kind == 1 and row.pin[2] > 0.999:
        x = row.xin[0] + row.pin[0]/row.pin[2] * (colimz - row.xin[2])
        y = row.xin[1] + row.pin[1]/row.pin[2] * (colimz - row.xin[2])
        histos['hcolspot1'].Fill(x,y)
        histos['hcolspec1'].Fill(np.log10(row.pin[3]/MeV))
        if x**2 + y**2 < colimr**2:
            histos['hcolspot2'].Fill(x,y)
            histos['hcolspec2'].Fill(np.log10(row.pin[3]/MeV))
    return histos

u = tview['det0'].declare_histograms("photon beam at collimator", beam_photon_hinit, beam_photon_hfill)        

In [8]:
#tview['det0'].fill_histograms()
n = tview['det0'].draw([['hcolspot1', 'hcolspec1'], ['hcolspot2', 'hcolspec2']], [['colz', ''], ['colz', '']], stats="ien")

## 3. Photons at collimator in coincidence with tags
This requires finding correlations between events in different trees: det0 for photons at the collimator, and det7 for electrons at the focal plane. To accomplish this, I start by making a pass through det7 and saving the event id, energy bin, and time for each hit in the focal plane. This table gets broadcast to all of the workers in the dask cluster in preparation for a pass through the det0 chain in search of photons within the same events to see what correlations exist between the tagger and the beam photons in energy and time. One key output from this scan is the tagging efficiency as a function of energy.

What I want to see here is the energy and times of all beam photons that pass through the collimator within a [-10,+10]ns window around the tagger hit time.

In [9]:
tags = {'tagm': tagm_tags, 'tagh': tagh_tags, 'det7': det7_tags}

In [10]:
def tag_photons_hinit():
    h = {}
    h['htagmtgam'] = ROOT.TH1D('htagmtgam', "tagged photon time for microscope tags", 200, 0, 20)
    h['htagmtgam'].GetXaxis().SetTitle("tbeam-tRF (ns)")
    h['htagmtgam'].GetYaxis().SetTitle("counts")
    h['htaghtgam'] = ROOT.TH1D('htaghtgam', "tagged photon time for hodoscope tags", 200, 0, 20)
    h['htaghtgam'].GetXaxis().SetTitle("tbeam-tRF (ns)")
    h['htaghtgam'].GetYaxis().SetTitle("counts")
    h['hdet7tgam'] = ROOT.TH1D('hdet7tgam', "tagged photon time for all focal plane tags", 200, 0, 20)
    h['hdet7tgam'].GetXaxis().SetTitle("tbeam-tRF (ns)")
    h['hdet7tgam'].GetYaxis().SetTitle("counts")
    h['htagmEvsE'] = ROOT.TH2D('htagmEvsE', "tagger focal plane z vs photon energy", 100, 730, 850, 100, 0, 4)
    h['htagmEvsE'].GetXaxis().SetTitle("focal plane z (cm)")
    h['htagmEvsE'].GetYaxis().SetTitle("E#gamma (GeV)")
    h['htaghEvsE'] = ROOT.TH2D('htaghEvsE', "tagger focal plane z vs photon energy", 100, 300, 1500, 100, 0, 4)
    h['htaghEvsE'].GetXaxis().SetTitle("focal plane z (cm)")
    h['htaghEvsE'].GetYaxis().SetTitle("E#gamma (GeV)")
    h['hdet7EvsE'] = ROOT.TH2D('hdet7EvsE', "tagger focal plane z vs photon energy", 100, 400, 1300, 100, 0, 4)
    h['hdet7EvsE'].GetXaxis().SetTitle("focal plane z (cm)")
    h['hdet7EvsE'].GetYaxis().SetTitle("E#gamma (GeV)")
    return h
def tag_photons_hfill(row, histos, colimz=7600, colimr=0.25, tsigma=0.5, ns=1e-9, Ibeam=1e-6):
    if row.kind == 1 and row.pin[2] > 0.999 and row.pin[3] > 1.0:
        electrons_per_4ns = Ibeam / 1.6e-19 * 4e-9
        bunches_in_sim = 5e8 / electrons_per_4ns
        bunches_tagm = bunches_in_sim / len(tags['tagm']) // 2
        bunch = np.random.randint(-bunches_tagm, bunches_tagm+1)
        histos['htagmtgam'].Fill(row.xin[3]/ns +  bunch * 4 + np.random.normal(scale=tsigma))
        if bunch == 0:
            randomtag = random.choice(list(tags['tagm']))
            for tagdata in tags['tagm'][randomtag]:
                histos['htagmEvsE'].Fill(tagdata[0], row.pin[3])
        bunches_tagh = 10 * bunches_in_sim / len(tags['tagh']) // 2
        for i in range(10):
            bunch = np.random.randint(-bunches_tagh, bunches_tagh+1)
            histos['htaghtgam'].Fill(row.xin[3]/ns +  bunch * 4 + np.random.normal(scale=tsigma))
            if bunch == 0:
                randomtag = random.choice(list(tags['tagh']))
                for tagdata in tags['tagh'][randomtag]:
                    histos['htaghEvsE'].Fill(tagdata[0], row.pin[3])
        bunches_det7 = 10 * bunches_in_sim / len(tags['det7']) // 2
        for i in range(10):
            bunch = np.random.randint(-bunches_det7, bunches_det7+1)
            histos['hdet7tgam'].Fill(row.xin[3]/ns +  bunch * 4 + np.random.normal(scale=tsigma))
            if bunch == 0:
                randomtag = random.choice(list(tags['det7']))
                for tagdata in tags['det7'][randomtag]:
                    histos['htaghEvsE'].Fill(tagdata[0], row.pin[3])
        evid = f"{row.GetFileNumber()}:{row.eventno}"
        if evid in tags['tagm']:
            histos['htagmtgam'].Fill(row.xin[3]/ns + np.random.normal(scale=tsigma))
            for tagdata in tags['tagm'][evid]:
                histos['htagmEvsE'].Fill(tagdata[0], row.pin[3])
        if evid in tags['tagh']:
            histos['htaghtgam'].Fill(row.xin[3]/ns + np.random.normal(scale=tsigma))
            for tagdata in tags['tagh'][evid]:
                histos['htaghEvsE'].Fill(tagdata[0], row.pin[3])
        if evid in tags['det7']:
            histos['hdet7tgam'].Fill(row.xin[3]/ns + np.random.normal(scale=tsigma))
            for tagdata in tags['det7'][evid]:
                histos['hdet7EvsE'].Fill(tagdata[0], row.pin[3])
    return histos

u = tview['det0'].declare_histograms("tagged photons", tag_photons_hinit, tag_photons_hfill)        

In [11]:
tview['det0'].fill_histograms(tags=tags)
u = tview['det0'].draw([['htagmtgam', 'htagmEvsE'], ['htaghtgam', 'htaghEvsE'], ['hdet7tgam', 'hdet7EvsE']],
                       [['', 'colz'], ['', 'colz'], ['', 'colz']], stats="ien")

fill_histograms read a total of 997 tree files, 498962168.0 records
